<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>Encoder-Decoder with Attention</H1></u></center>

In [ ]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess)

In [ ]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal

In [ ]:
# function to generate random integer values between 0 and n. The number of timestpes is: length
def gen_seq(length, n):
    return [randint(0, n-1) for _ in range(length)]

## Generate sequence:

In [ ]:
sequence = gen_seq(6, 30)
sequence

In [ ]:
def onehot_encoder(seq, n):
    encod = []
    for s in seq:
        v = [0 for _ in range(n)]
        v[s] = 1
        encod.append(v)
    return array(encod)

In [ ]:
def onehot_decoder(encod_seq):
    return [argmax(idx) for idx in encod_seq]

In [ ]:
onehot = onehot_encoder(sequence, 30)
print(onehot)

In [ ]:
decoded = onehot_decoder(onehot)
print(decoded)

In [ ]:
def generate_pair(n_in, n_out, n_total):
    # generating random sequences
    seq_in = gen_seq(n_in, n_total)
    seq_out = seq_in[:n_out] + [0 for _ in range(n_in-n_out)]
    
    X = onehot_encoder(seq_in, n_total)
    y = onehot_encoder(seq_out, n_total)
    
    # reshaping as 3D tensor
    X = X.reshape((1, X.shape[0], X.shape[1]))
    y = y.reshape((1, y.shape[0], y.shape[1]))
    return X,y

In [ ]:
X, y = generate_pair(6, 3, 30)
print('X=%s, y=%s' % (onehot_decoder(X[0]), onehot_decoder(y[0])))

In [ ]:
print(X.shape, y.shape)

## Encoder-Decoder without Attention

In [ ]:
n_features = 50
n_timesteps_in = 5
n_timesteps_out = 3

## Creating the model: 

In [ ]:
model = Sequential()

In [ ]:
model.add(CuDNNLSTM(150, input_shape=(n_timesteps_in, n_features)))

In [ ]:
model.add(RepeatVector(n_timesteps_in))

In [ ]:
model.add(CuDNNLSTM(150, return_sequences=True))

In [ ]:
model.add(TimeDistributed(Dense(n_features, activation='softmax')))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

## Training the model:

In [ ]:
for epoch in range(5000):
    X,y = generate_pair(n_timesteps_in, n_timesteps_out, n_features)
    model.fit(X, y, epochs=1, verbose=1)

## Testing the model:

In [ ]:
epochs = 100
correct = 0

In [ ]:
# Testing the model with new 100 new randomly generated integer sequences
for _ in range(epochs):
    X,y = generate_pair(n_timesteps_in, n_timesteps_out, n_features)
    pred = model.predict(X)
    if array_equal(onehot_decoder(y[0]), onehot_decoder(pred[0])):
        correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(epochs)*100.0))

## Checking samples:

In [ ]:
#checking 20 examples of expected output sequences and predictions
for _ in range(20):
    X,y = generate_pair(n_timesteps_in, n_timesteps_out, n_features)
    pred = model.predict(X)
    print('Expected:', onehot_decoder(y[0]), 'Predicted', onehot_decoder(pred[0]))

## Encoder-Decoder with Attention Layer

In [ ]:
from attention_decoder import AttentionDecoder

## Creating the model with Attention:

In [ ]:
model_att = Sequential()

In [ ]:
model_att.add(CuDNNLSTM(150, input_shape=(n_timesteps_in, n_features), return_sequences=True))

In [ ]:
model_att.add(CuDNNLSTM(150, return_sequences=True))

In [ ]:
model_att.add(CuDNNLSTM(150, return_sequences=True))

In [ ]:
model_att.add(AttentionDecoder(150, n_features))

In [ ]:
model_att.summary()

In [ ]:
model_att.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
for epoch in range(5000):
    X, y = generate_pair(n_timesteps_in, n_timesteps_out, n_features)
    model_att.fit(X, y, epochs=1, verbose=1)

## Testing the attention model:

In [ ]:
for _ in range(epochs):
    X, y = generate_pair(n_timesteps_in, n_timesteps_out, n_features)
    pred_att = model_att.predict(X)
    if array_equal(onehot_decoder(y[0]), onehot_decoder(pred_att[0])):
        correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(epochs)*100.0))

## Checking samples:

In [ ]:
for _ in range(20):
    X,y = generate_pair(n_timesteps_in, n_timesteps_out, n_features)
    pred = model_att.predict(X)
    print('Expected:', onehot_decoder(y[0]), 'Predicted', onehot_decoder(pred[0]))